<a href="https://colab.research.google.com/github/tinetor/Covid/blob/master/fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open("taylor_swift_1000_qa.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
import datasets
taylor_swift_dataset = "lamini/taylor_swift"
bts_dataset = "lamini/bts"
open_llms = "lamini/open_llms"
dataset_swiftie = datasets.load_dataset(taylor_swift_dataset)
print(dataset_swiftie["train"][1])
def tokenize_function(example):
    # O prompt é 'question'
    # A resposta é 'answer'
    full_prompt = f"Q: {example['question']}\nA: {example['answer']}"
    tokens = tokenizer(
        full_prompt,
        truncation=True,
        max_length=512,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens
tokenized_dataset_swiftie = dataset_swiftie.map(
    tokenize_function,
    batched=False,     # Cada item é processado individualmente
    remove_columns=dataset_swiftie["train"].column_names,  # Remove input/output cru se quiser
)

{'question': 'What is the most popular Taylor Swift song among millennials? How does this song relate to the millennial generation? What is the significance of this song in the millennial culture?', 'answer': 'Taylor Swift\'s "Shake It Off" is the most popular song among millennials. This song relates to the millennial generation as it is an anthem of self-acceptance and embracing one\'s individuality. The song\'s message of not letting others bring you down and to just dance it off resonates with the millennial culture, which is often characterized by a strong sense of individuality and a rejection of societal norms. Additionally, the song\'s upbeat and catchy melody makes it a perfect fit for the millennial generation, which is known for its love of pop music.', 'input_ids': [1276, 310, 253, 954, 4633, 11276, 24619, 4498, 2190, 24933, 8075, 32, 1359, 1057, 436, 4498, 14588, 281, 253, 24933, 451, 5978, 32, 1737, 310, 253, 8453, 273, 436, 4498, 275, 253, 24933, 451, 4466, 32, 37979, 24

Map:   0%|          | 0/783 [00:00<?, ? examples/s]

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset_swiftie

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 783
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 87
    })
})

In [ ]:
pip install  trl
pip install bitsandbytes

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch


model_id = 'Qwen/Qwen3-0.6B'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


device = "cuda:0"
tokenizer = AutoTokenizer.from_pretrained(model_id)



from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
# 1. Quebrar em pares Q&A
pairs = raw_text.strip().split("\n\n")

# 2. Criar lista com coluna 'text' no formato chat-style
data = []
for pair in pairs:
    lines = pair.strip().split("\n")
    if len(lines) >= 2:
        question = lines[0].replace("Q: ", "").strip()
        answer = lines[1].replace("A: ", "").strip()
        text = f"user: {question}\nassistant: {answer}"
        data.append({"text": text})

# 3. Criar Hugging Face Dataset
from datasets import Dataset, DatasetDict
dataset = Dataset.from_list(data)

# 4. Dividir em treino e teste
dataset = dataset.train_test_split(test_size=0.1)
dataset = DatasetDict({
    "train": dataset["train"],
    "test": dataset["test"]
})
# 5. Definir função para tokenizar e padronizar entrada
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

# 8. Mapear a tokenização no dataset (gera colunas input_ids, attention_mask)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

os.environ["WANDB_DISABLED"] = "true"

# Configs
model_id = 'nvidia/Llama-3.1-Nemotron-Nano-8B-v1'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Modelo base
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0}
)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# LoRA config
peft_config = LoraConfig(
    r=6,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

# Verifica
print_trainable_parameters(model)

# Trainer
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    max_steps=100,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="outputs",
    optim="paged_adamw_8bit"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_swiftie['train'],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False
trainer.train()


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 14.74 GiB of which 912.12 MiB is free. Process 6095 has 13.72 GiB memory in use. Of the allocated memory 11.53 GiB is allocated by PyTorch, and 2.05 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
messages = [{"role": "user", "content": "Can you tell me something negative about Taylor Swift?"}]
model.gradient_checkpointing_disable()
model.config.use_cache = True
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to(trainer.model.device)
outputs = trainer.model.generate(input_ids=input_ids, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
Can you tell me something negative about Taylor Swift?

<think>
Okay, the user asked for a negative thing about Taylor Swift. I need to provide a negative review. First, I should mention her popularity. She's a huge star, so that's a common point. Then, I can talk about her music style. Her music has a lot of emotional depth, so that's a good point. I should mention how she's a trendsetter. She's always pushing boundaries, so that's a good angle. I can also mention her


In [ ]:
trainer.model

In [ ]:
messages = [
    {"role": "system", "content": "You are an assistant. Answer only in English."},
    {"role": "user", "content": "Can you tell me the list of Taylor Swift's ex-boyfriends?"}
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

inputs = tokenizer(prompt, return_tensors="pt").to(trainer.model.device)
outputs = trainer.model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<|im_start|>system
You are an assistant. Answer only in English.<|im_end|>
<|im_start|>user
Can you tell me the list of Taylor Swift's ex-boyfriends?<|im_end|>
<|im_start|>assistant

system
You are an assistant. Answer only in English.
user
Can you tell me the list of Taylor Swift's ex-boyfriends?
assistant
<think>
Okay, the user is asking for the list of Taylor Swift's ex-boyfriends. I need to confirm if I can provide that information. Taylor Swift's ex-boyfriends are: Taylor Swift, Taylor Swift, Taylor Swift, Taylor Swift,


In [ ]:
messages = [{"role": "user", "content": "When was Taylor Swift born?"}]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print("🚩 Prompt final:\n", prompt)


🚩 Prompt final:
 <|im_start|>user
When was Taylor Swift born?<|im_end|>
<|im_start|>assistant

